### Альвицов Данил Сергеевич БВТ2204
### Курсовая Работа

#### Импорты 

In [8]:
import random
from datetime import datetime, timedelta

#### Константы

In [ ]:
WEEKDAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
WEEKENDS = ['Saturday', 'Sunday']
SHIFT_8_HOURS = timedelta(hours=8)
SHIFT_12_HOURS = timedelta(hours=12)

#### 1. Алгоритм Brute-Force 

In [ ]:
class Vodila:
    def __init__(self, driver_id, shift_type):
        self.driver_id = driver_id
        self.shift_type = shift_type
        self.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
        self.breaks = {day: [] for day in WEEKDAYS + WEEKENDS}

class Bus:
    def __init__(self, bus_id, bus_index):
        self.bus_id = bus_id
        self.schedule = {day: [] for day in WEEKDAYS + WEEKENDS}
        self.drivers = []
        self.bus_index = bus_index

buses = [Bus(f"Bus_{i+1}", i) for i in range(8)]

driver_count = 0
drivers = []

rest_day_counter = {}

def create_time_blocks(start_time, shift_duration):
    start = datetime.strptime(start_time, "%H:%M")
    end = start + shift_duration
    return start, end

def assign_breaks(driver, start, end, break_type):
    breaks = []
    if break_type == "15-min": 
        current = start + timedelta(minutes=105)

        if driver.shift_type == "12-hour":
            break_duration = timedelta(minutes=18)
            while current + break_duration < end:
                breaks.append((current.strftime("%H:%M"), (current + break_duration).strftime("%H:%M"), "15-min"))
                current += timedelta(minutes=120)
        else:
            break_duration = timedelta(minutes=15)
            while current + break_duration < end:
                breaks.append((current.strftime("%H:%M"), (current + break_duration).strftime("%H:%M"), "15-min"))
                current += timedelta(minutes=120)

    elif break_type == "1-hour":
        if start.strftime("%H:%M") == "09:00":
            lunch_start = datetime.strptime("14:15", "%H:%M")
            lunch_end = lunch_start + timedelta(minutes=45)
        else:
            lunch_start = datetime.strptime("13:00", "%H:%M")
            lunch_end = lunch_start + timedelta(hours=1)

        if start <= lunch_start < end:
            breaks.append((lunch_start.strftime("%H:%M"), lunch_end.strftime("%H:%M"), "1-hour"))
        elif lunch_start <= start < lunch_end:
            breaks.append((start.strftime("%H:%M"), (start + timedelta(hours=1)).strftime("%H:%M"), "1-hour"))

    return breaks

def allocate_8hr_drivers():
    global driver_count

    for bus_index, bus in enumerate(buses):
        if bus_index < 4:
            shift_start_time = "06:00"
        else:
            shift_start_time = "09:00"

        driver_count += 1
        driver = Vodila(f"Driver_{driver_count}_8hr", "8-hour")
        drivers.append(driver)

        for day in WEEKDAYS:
            staggered_start_time = (
                datetime.strptime(shift_start_time, "%H:%M") + timedelta(minutes=(bus_index % 4) * 15)
            ).strftime("%H:%M")
            start, end = create_time_blocks(staggered_start_time, SHIFT_8_HOURS)

            if "09:00" in staggered_start_time:
                breaks = assign_breaks(driver, start, end, "1-hour")
            else:
                breaks = assign_breaks(driver, start, end, "15-min")

            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

def allocate_12hr_drivers():
    global driver_count
    rest_day_counter = {}

    for bus_index, bus in enumerate(buses):
        if bus_index < 4:  
            for i in range(3):
                driver_count += 1
                driver = Vodila(f"Driver_{driver_count}_12hr", "12-hour")
                bus.drivers.append(driver)
                drivers.append(driver)
                rest_day_counter[driver.driver_id] = 0
        else:
            for i in range(3):
                driver_count += 1
                driver = Vodila(f"Driver_{driver_count}_12hr", "12-hour")
                bus.drivers.append(driver)
                drivers.append(driver)
                rest_day_counter[driver.driver_id] = 0 

    for bus in buses:
        driver_index = 0

        for day in WEEKDAYS + WEEKENDS:
            while rest_day_counter[bus.drivers[driver_index].driver_id] > 0:
                rest_day_counter[bus.drivers[driver_index].driver_id] -= 1
                driver_index = (driver_index + 1) % len(bus.drivers)

            driver = bus.drivers[driver_index]

            if day in WEEKDAYS:
                if bus.bus_index < 4:
                    staggered_start_time = (datetime.strptime("14:00", "%H:%M") + timedelta(minutes=bus.bus_index * 15)).strftime("%H:%M")
                else:
                    staggered_start_time = (datetime.strptime("17:00", "%H:%M") + timedelta(minutes=(bus.bus_index - 4) * 15)).strftime("%H:%M")
            else:
                staggered_start_time = (datetime.strptime("06:00", "%H:%M") + timedelta(minutes=bus.bus_index * 18)).strftime("%H:%M")

            start, end = create_time_blocks(staggered_start_time, SHIFT_12_HOURS)

            if day in WEEKDAYS + ["Saturday", "Sunday"]:
                if start.hour == 6 and start.minute == 0:
                    breaks = assign_breaks(driver, start, end, "1-hour")
                else:
                    breaks = assign_breaks(driver, start, end, "15-min")
            else:
                breaks = assign_breaks(driver, start, end, "15-min")

            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

            rest_day_counter[driver.driver_id] = 2

            driver_index = (driver_index + 1) % len(bus.drivers)

def allocate_weekend_12hr_drivers():
    global driver_count

    for _, bus in enumerate(buses):
        for day in WEEKENDS:
            driver_count += 1
            driver = Vodila(f"Driver_{driver_count}_12hr", f"12-hour Weekend {day}")
            bus.drivers.append(driver)
            drivers.append(driver)

            start_time = (datetime.strptime("18:03", "%H:%M") + timedelta(minutes=bus.bus_index * 18)).strftime("%H:%M")
            start, end = create_time_blocks(start_time, SHIFT_12_HOURS)

            breaks = assign_breaks(driver, start, end, "15-min")
            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

allocate_8hr_drivers()
allocate_12hr_drivers()
allocate_weekend_12hr_drivers()


print(f"Drivers Used: {driver_count}")

print("Bus Schedule:")
for bus in buses:
    print(f"\n{bus.bus_id} Weekly Schedule:")
    for day in WEEKDAYS + WEEKENDS:
        if bus.schedule[day]:
            print(f"  {day}:")
            for shift in bus.schedule[day]:
                start_time = shift[0].strftime("%H:%M")
                end_time = shift[1].strftime("%H:%M")
                driver_id = shift[2]
                print(f"    Time: {start_time} - {end_time}, Driver: {driver_id}")

print("\nDriver Schedule:")
for driver in drivers:
    print(f"\n{driver.driver_id} ({driver.shift_type}) Weekly Schedule:")
    for day in WEEKDAYS + WEEKENDS:
        if driver.schedule[day]:
            print(f"  {day}:")
            for shift in driver.schedule[day]:
                start_time = shift[0].strftime("%H:%M")
                end_time = shift[1].strftime("%H:%M")
                bus_id = shift[2]
                print(f"    Time: {start_time} - {end_time}, Bus: {bus_id}")
            if driver.breaks[day]:
                print("    Breaks:")
                for b in driver.breaks[day]:
                    print(f"      {b[0]} - {b[1]}")


Drivers Used: 48
Bus Schedule:

Bus_1 Weekly Schedule:
  Monday:
    Time: 06:00 - 14:00, Driver: Driver_1_8hr
    Time: 14:00 - 02:00, Driver: Driver_9_12hr
  Tuesday:
    Time: 06:00 - 14:00, Driver: Driver_1_8hr
    Time: 14:00 - 02:00, Driver: Driver_10_12hr
  Wednesday:
    Time: 06:00 - 14:00, Driver: Driver_1_8hr
    Time: 14:00 - 02:00, Driver: Driver_11_12hr
  Thursday:
    Time: 06:00 - 14:00, Driver: Driver_1_8hr
    Time: 14:00 - 02:00, Driver: Driver_9_12hr
  Friday:
    Time: 06:00 - 14:00, Driver: Driver_1_8hr
    Time: 14:00 - 02:00, Driver: Driver_10_12hr
  Saturday:
    Time: 06:00 - 18:00, Driver: Driver_11_12hr
    Time: 18:03 - 06:03, Driver: Driver_33_12hr
  Sunday:
    Time: 06:00 - 18:00, Driver: Driver_9_12hr
    Time: 18:03 - 06:03, Driver: Driver_34_12hr

Bus_2 Weekly Schedule:
  Monday:
    Time: 06:15 - 14:15, Driver: Driver_2_8hr
    Time: 14:15 - 02:15, Driver: Driver_12_12hr
  Tuesday:
    Time: 06:15 - 14:15, Driver: Driver_2_8hr
    Time: 14:15 - 02:15

### 2. Генетический Алгоритм

In [11]:
driver_count = 0
drivers = []

rest_days_counter = {}

def initialize_population(pop_size):
    population = []
    for _ in range(pop_size):
        individual = create_random_schedule()
        population.append(individual)
    return population

def create_random_schedule():
    global driver_count
    schedule = []
    rest_day_counter = {driver.driver_id: 0 for driver in drivers if driver.shift_type == "12-hour"}

    for bus_index, bus in enumerate(buses):
        for day in WEEKDAYS:
            shift_type = "8-hour"
            shift_start_time = "06:00" if bus_index < 4 else "09:00"

            staggered_start_time = (
                datetime.strptime(shift_start_time, "%H:%M") + timedelta(minutes=(bus_index % 4) * 15)
            ).strftime("%H:%M")

            suitable_drivers = [d for d in drivers if d.shift_type == "8-hour"]
            driver = random.choice(suitable_drivers)

            shift_hours = SHIFT_8_HOURS
            start, end = create_time_blocks(staggered_start_time, shift_hours)

            breaks = assign_breaks(driver, start, end, "1-hour" if "09:00" in staggered_start_time else "15-min")

            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))

            schedule.append((bus.bus_id, day, start, end, driver.driver_id))

    for bus_index, bus in enumerate(buses):
        for day in WEEKDAYS + WEEKENDS:
            if day in WEEKENDS:
                if bus_index < 4:
                    shift_type = "12-hour (Weekend Saturday)" if day == "Saturday" else "12-hour (Weekend Sunday)"
                    shift_start_time = "06:00"
                else:
                    shift_type = f"12-hour (Weekend {day})"
                    shift_start_time = "18:03"
            elif day in WEEKDAYS:
                shift_type = "12-hour"
                shift_start_time = "14:00" if bus_index < 4 else "17:00"

            staggered_start_time = (
                datetime.strptime(shift_start_time, "%H:%M") + timedelta(minutes=(bus_index % 4) * 15)
            ).strftime("%H:%M")

            if shift_type == "12-hour (Weekend Saturday)":
                suitable_drivers = [d for d in drivers if d.shift_type == "12-hour (Weekend Saturday)"]
            elif shift_type == "12-hour (Weekend Sunday)":
                suitable_drivers = [d for d in drivers if d.shift_type == "12-hour (Weekend Sunday)"]
            elif shift_type == "12-hour":
                suitable_drivers = [d for d in drivers if d.shift_type == "12-hour" and rest_day_counter.get(d.driver_id, 0) == 0]

            if suitable_drivers:
                driver = random.choice(suitable_drivers)
            else:
                continue

            shift_hours = SHIFT_12_HOURS
            start, end = create_time_blocks(staggered_start_time, shift_hours)

            if day in WEEKENDS and start.hour == 6 and start.minute == 0:
                breaks = assign_breaks(driver, start, end, "1-hour")
            else:
                breaks = assign_breaks(driver, start, end, "15-min")

            driver.breaks[day] = breaks
            driver.schedule[day].append((start, end, bus.bus_id))
            bus.schedule[day].append((start, end, driver.driver_id))
            schedule.append((bus.bus_id, day, start, end, driver.driver_id))

    return schedule

def fitness_function(schedule):
    penalty = 0
    driver_shifts = {driver.driver_id: {day: [] for day in WEEKDAYS + WEEKENDS} for driver in drivers}

    for _, day, start, end, driver_id in schedule:
        driver_shifts[driver_id][day].append((start, end))

    for driver_id, shifts in driver_shifts.items():
        for day, day_shifts in shifts.items():
            day_shifts.sort()
            for i in range(len(day_shifts)):
                for j in range(i + 1, len(day_shifts)):
                    if day_shifts[i][1] > day_shifts[j][0] and day_shifts[i][0] < day_shifts[j][1]:
                        penalty += 10

    for driver_id, shifts in driver_shifts.items():
        work_days = sum(1 for day, day_shifts in shifts.items() if day_shifts)
        if work_days > 5:
            penalty += 20

    return penalty

def selection(population, fitness_scores):
    selected = random.choices(population, weights=[max(fitness_scores) - score for score in fitness_scores], k=2)
    return selected

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(individual, mutation_rate):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = create_random_schedule()[i]
    return individual

def genetic_algorithm(pop_size, generations, mutation_rate):
    population = initialize_population(pop_size)
    for _ in range(generations):
        fitness_scores = [fitness_function(individual) for individual in population]

        new_population = []
        for _ in range(pop_size // 2):
            parent1, parent2 = selection(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([mutate(child1, mutation_rate), mutate(child2, mutation_rate)])
        population = new_population

    best_individual = min(population, key=fitness_function)
    return best_individual

def allocate_drivers():
    global driver_count
    for _, bus in enumerate(buses):
        for _ in range(3):
            driver_count += 1
            driver = Vodila(f"Driver_{driver_count}_12hr", "12-hour")
            bus.drivers.append(driver)
            drivers.append(driver)

        for _ in range(2):
            driver_count += 1
            driver = Vodila(f"Driver_{driver_count}_8hr", "8-hour")
            bus.drivers.append(driver)
            drivers.append(driver)

        driver_count += 1
        driver = Vodila(f"Driver_{driver_count}_12hr", "12-hour (Weekend Saturday)")
        bus.drivers.append(driver)
        drivers.append(driver)

        driver_count += 1
        driver = Vodila(f"Driver_{driver_count}_12hr", "12-hour (Weekend Sunday)")
        bus.drivers.append(driver)
        drivers.append(driver)

allocate_drivers()

def convert_schedule_to_format(schedule):
    bus_schedule = {bus.bus_id: {day: [] for day in WEEKDAYS + WEEKENDS} for bus in buses}
    driver_schedule = {driver.driver_id: {day: [] for day in WEEKDAYS + WEEKENDS} for driver in drivers}

    for bus_id, day, start, end, driver_id in schedule:
        bus_schedule[bus_id][day].append((start, end, driver_id))
        driver_schedule[driver_id][day].append((start, end, bus_id))

    return bus_schedule, driver_schedule

best_schedule = genetic_algorithm(pop_size=50, generations=100, mutation_rate=0.01)

bus_schedule, driver_schedule = convert_schedule_to_format(best_schedule)

print(f"Drivers Used: {driver_count}")

print("Bus Schedule:")
for bus_id, schedule in bus_schedule.items():
    print(f"\n{bus_id} Weekly Schedule:")
    for day, shifts in schedule.items():
        if shifts:
            print(f"  {day}:")
            for shift in shifts:
                start_time = shift[0].strftime("%H:%M")
                end_time = shift[1].strftime("%H:%M")
                driver_id = shift[2]
                print(f"    Time: {start_time} - {end_time}, Driver: {driver_id}")

print("\nDriver Schedule:")
for driver_id, schedule in driver_schedule.items():
    print(f"\n{driver_id} Weekly Schedule:")
    for day, shifts in schedule.items():
        if shifts:
            print(f"  {day}:")
            for shift in shifts:
                start_time = shift[0].strftime("%H:%M")
                end_time = shift[1].strftime("%H:%M")
                bus_id = shift[2]
                print(f"    Time: {start_time} - {end_time}, Bus: {bus_id}")

            for driver in drivers:
                if driver.driver_id == driver_id:
                    if driver.breaks[day]:
                        lunch_breaks = [
                            f"{b[0]}-{b[1]}" for b in driver.breaks[day] if b[2] == "1-hour"
                        ]
                        short_breaks = [
                            f"{b[0]}-{b[1]}" for b in driver.breaks[day] if b[2] == "15-min"
                        ]

                        if lunch_breaks or short_breaks:
                            print("    Breaks:")

                            if lunch_breaks:
                                print("      Lunch Breaks:")
                                for b in lunch_breaks:
                                    print(f"        {b}")

                            if short_breaks:
                                print("      Short Breaks:")
                                for b in short_breaks:
                                    print(f"        {b}")



Drivers Used: 56
Bus Schedule:

Bus_1 Weekly Schedule:
  Monday:
    Time: 06:00 - 14:00, Driver: Driver_39_8hr
    Time: 14:00 - 02:00, Driver: Driver_31_12hr
  Tuesday:
    Time: 06:00 - 14:00, Driver: Driver_47_8hr
    Time: 14:00 - 02:00, Driver: Driver_2_12hr
  Wednesday:
    Time: 06:00 - 14:00, Driver: Driver_25_8hr
    Time: 14:00 - 02:00, Driver: Driver_15_12hr
  Thursday:
    Time: 06:00 - 14:00, Driver: Driver_5_8hr
    Time: 14:00 - 02:00, Driver: Driver_17_12hr
  Friday:
    Time: 06:00 - 14:00, Driver: Driver_39_8hr
    Time: 14:00 - 02:00, Driver: Driver_38_12hr
  Saturday:
    Time: 06:00 - 18:00, Driver: Driver_41_12hr
  Sunday:
    Time: 06:00 - 18:00, Driver: Driver_28_12hr

Bus_2 Weekly Schedule:
  Monday:
    Time: 06:15 - 14:15, Driver: Driver_46_8hr
    Time: 14:15 - 02:15, Driver: Driver_30_12hr
  Tuesday:
    Time: 06:15 - 14:15, Driver: Driver_18_8hr
    Time: 14:15 - 02:15, Driver: Driver_22_12hr
  Wednesday:
    Time: 06:15 - 14:15, Driver: Driver_26_8hr
   